In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [98]:
class ExtendableTS(object):
    def __init__(self, pkeys:list=None, time_col:str="ts"):
        self.df = pd.DataFrame()
        self.pkeys = pkeys if pkeys else ["ts"]
        self.time_col = time_col

    def get_max_time(self):
        if self.df.empty:
            return None
        return self.df[self.time_col].max()

    def assign_or_extend(self, df):
        self.df = pd.concat([self.df, df]).drop_duplicates(subset=self.pkeys, keep="first")

    def cut_off_data_less_than(self, ts):
        if self.df.empty:
            return
        self.df = self.df[self.df[self.time_col] >= ts]


In [66]:
def make_data_with_ts(start:int, end:int, step:int):

    
    start = round(start / step) * step
    end = round(end / step) * step

    _ = np.arange(start, end, step)

    df = pd.DataFrame({
        'ts': list(_),
        'value': np.random.randn(len(_))
    })
    df["time"] = pd.to_datetime(df["ts"], unit='s')

    return df

### process

In [67]:
now = datetime.datetime(2025,7,20).timestamp()
now, datetime.datetime.fromtimestamp(now)

(1752966000.0, datetime.datetime(2025, 7, 20, 0, 0))

In [99]:
ets = ExtendableTS(pkeys=["ts"], time_col="ts")

_ = make_data_with_ts(
    start = now - 5 * 60 * 60,
    end = now - 3 * 60 * 60,
    step = 60 * 60
)
ets.assign_or_extend(_)

_ = make_data_with_ts(
    start = ets.get_max_time() - 1,
    end = now - 2 * 60 * 60,
    step = 60 * 60
)
ets.assign_or_extend(_)

ets.cut_off_data_less_than(now - 4 * 60 * 60)

In [100]:
ets.df

,ts,value,time
1,1752951600,-0.897426,2025-07-19 19:00:00
1,1752955200,-1.680630,2025-07-19 20:00:00
